In [ ]:
import pandas as pd
df = pd.read_parquet('catB_train.parquet')
#print(df.head())
#print(df.columns)
#print(df.shape)
print(df.describe())

In [ ]:
import os
print(os.getcwd())
new_directory = 'D:/Sou/nus/Y3S2/Datathon/nus-datathon-24-category-b'
os.chdir(new_directory)
print(os.getcwd())

In [ ]:
numerical_columns = df.select_dtypes(include=['number']).columns

# Print the names of numerical columns
print("Numerical Columns:")
print(numerical_columns)